In [103]:
from selenium.webdriver import Chrome
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pandas as pd
data = {'Product ID*':[],'Attribute (Name:Type:Position)*':[],'Value (Value:Position)*':[],'Reference':[],'Wholesale price':[], 'Image URLs (x,y,z...)':[]}
df_atributos = pd.DataFrame(data)
dat = {'Product ID*':[],'Active (0/1)':[],'Name *':[],'Categories (x,y,z...)':[],'Price tax excluded':[], 'Reference #':[], 'Description':[], 'Tags (x,y,z...)':[], 'Image URLs (x,y,z...)':[]}
df_productos = pd.DataFrame(dat)
driver = Chrome()
driver.get('https://tegler.es/productos/iluminacion/')

In [ ]:
#Quitar el IVA a los precios
def impactPrice():
    priceOG = float(driver.find_element_by_class_name('price').text.replace(',', '.').replace('Iva Incl.','').replace(' € ','').replace('Desde: ',''))/ 1.21
    priceVAR = (driver.find_element_by_class_name('woocommerce-variation-price').text.replace(',', '.').replace('Iva Incl.','').replace(' €\n',''))
    if priceVAR == '':
        priceVAR = 0
    return round(float(priceVAR)/1.21-priceOG, 2)

In [ ]:
#Funciones de utilidad para estructurar el código
def atributos():
    #mamparas 0,0 otro 1,0
    colores = driver.find_elements_by_class_name('swatch.swatch-image')
    if colores != []:
        try:
            n = driver.find_elements_by_class_name('label')[1]
            desplegable = Select(driver.find_element_by_id(n.find_element_by_tag_name('label').get_attribute('for')))
            
            return (colores, desplegable)
        except:
            return (colores, None)
    else:
        try:
            n = driver.find_elements_by_class_name('label')[0]
            desplegable = Select(driver.find_element_by_id(n.find_element_by_tag_name('label').get_attribute('for')))
            return (None,desplegable)
        except:
            return (None,None)
        
def recorrer_desplegable(desplegable, df_atributos, n, color = None):
    elementos = (len(desplegable.options)-1)
    for i in range(elementos):
        desplegable.select_by_index(i+1)
        time.sleep(3)
        medida = desplegable.options[i+1].text
        sku = driver.find_element_by_class_name('sku').text
        impact = impactPrice()
        if color is not None:
            pho = color.find_element_by_tag_name('img').get_attribute('src')
            c = color.text
            val= f'{c}:0, {medida}:1'
            atr = 'Color:color:0, Medida:select:1'
            linea_atributos = {'Product ID*':n,'Attribute (Name:Type:Position)*':atr,'Value (Value:Position)*':val,'Reference':sku,'Wholesale price':impact,'Image URLs (x,y,z...)':pho}
            df_atributos = df_atributos.append(linea_atributos, ignore_index = True)
        else:
            val= f'{medida}:0'
            atr = 'Medida:select:0'
            linea_atributos = {'Product ID*':n,'Attribute (Name:Type:Position)*':atr,'Value (Value:Position)*':val,'Reference':sku,'Wholesale price':impact, 'Image URLs (x,y,z...)':None}
            df_atributos = df_atributos.append(linea_atributos, ignore_index = True)
        break
    return(df_atributos)
            
def recorrer_atributos(df_atributos, n):
    colores, desplegable = atributos()
    if colores is not None:
        if desplegable is not None:
            i = 0
            for valor in colores:
                time.sleep(2)
                valor.click()
                atr = recorrer_desplegable(desplegable, df_atributos, n, colores[i])
                i += 1
                df_atributos = atr
                break
            return df_atributos
        else:
            for valor in colores:
                time.sleep(2)
                valor.click()
                time.sleep(1)
                pho = valor.find_element_by_tag_name('img').get_attribute('src')
                sku = driver.find_element_by_class_name('sku').text
                print('preparado...')
                #------------------------------
                priceOG = float(driver.find_element_by_class_name('price').text.replace(',', '.').replace('Iva Incl.','').replace(' € ','').replace('Desde: ',''))/ 1.21
                print(1)
                price = float(driver.find_element_by_class_name('woocommerce-variation-price').text.replace(',', '.').replace('Iva Incl.','').replace(' €\n',''))/1.21
                print(2)
                if price == '':
                    price = 0
                impact = round(price-priceOG, 2)
                #--------------------------------------------
                print('impact')
                val= f'{valor.text}:0'
                atr = 'Color:color:0'
                linea_atributos = {'Product ID*':n,'Attribute (Name:Type:Position)*':atr,'Value (Value:Position)*':val,'Reference':sku,'Wholesale price':impact,'Image URLs (x,y,z...)':pho}
                df_atributos = df_atributos.append(linea_atributos, ignore_index = True)
                break
            return df_atributos
    else:
        if desplegable is not None:
            atr = recorrer_desplegable(desplegable,df_atributos, n)
            df_atributos = atr
            return df_atributos
            
        else:
            return df_atributos

def recorrer_producto(df_productos, n):
    print(n)
    photos = driver.find_elements_by_class_name('woocommerce-product-gallery__image')
    str_photos = ''
    for photo in photos:
        str_photos = str_photos + ',' + photo.find_element_by_tag_name('img').get_attribute('src')
    str_photos = str_photos[1:]
    title = driver.find_element_by_class_name('product_title.entry-title').text
    try:
        sku = driver.find_element_by_class_name('sku').text
    except:
        sku = None
    try:
        categories = driver.find_element_by_class_name('posted_in').text.replace(', ', ',').replace('CATEGORÍAS: ', '')
    except:
        categories = None
    try:
        tags = driver.find_element_by_class_name('tagged_as').text.replace(', ', ',').replace('ETIQUETAS: ', '')
    except:
        tags= None
    desc = driver.find_element_by_class_name('woocommerce-Tabs-panel.woocommerce-Tabs-panel--description.panel.entry-content.wc-tab').text
    price = driver.find_element_by_class_name('price').text.replace('Iva Incl.','').replace(' € ','').replace('Desde: ','')
    if len(price) == 8:
        print(price)
        price = price.replace('.','').replace(',', '.')
        print(price)
    else:
        price = price.replace(',', '.')
    
    price = round(float(price)/1.21,2)
    producto = {'Product ID*':n,'Active (0/1)':1,'Name *':title,'Categories (x,y,z...)':categories,'Price tax excluded':price, 'Reference #':sku, 'Description':desc, 'Tags (x,y,z...)':tags, 'Image URLs (x,y,z...)':str_photos}
    df_productos = df_productos.append(producto, ignore_index=True)
    return df_productos 

def recorrer_superproductos(df_productos, df_atributos, n):
    productos = driver.find_elements_by_class_name('image-wrapper')
    f = 0
    i = 0
    for valor in range(len(productos)):
        print(f'producto{n}')
        i += 1
        n += 1
        elem = driver.find_elements_by_class_name('image-wrapper')[i-1]
        elem.click()
        time.sleep(2)
        df_mas = recorrer_producto(df_productos, n)
        if df_mas is not None:
            if atributos() == (None, None):
                #df_atr = recorrer_atributos(df_atributos, n)
                #df_atributos = df_atr
                df_productos = df_mas
        driver.back()
        if i %3 == 0:
            f += 400
            driver.execute_script(f"window.scrollTo(0, {f})")
        time.sleep(3)
    return(df_productos, df_atributos, n)

def recorrer_supersuper(df_productos, df_atributos):
    i = 0
    n = int(0)
    f = 0
    driver.maximize_window()
    for valor in range(len(driver.find_elements_by_class_name('woocommerce-loop-category__title'))):
        try:
            elem = driver.find_elements_by_class_name('woocommerce-loop-category__title')[i]
            f += 50
            driver.execute_script(f"window.scrollTo(0, {f})")
            i += 1
            elem.click()
            try:
                df_mas_productos , df_mas_atributos, nuevo_n= recorrer_superproductos(df_productos, df_atributos, n)
                df_productos, df_atributos, n = df_mas_productos , df_mas_atributos, nuevo_n
            except:
                return(df_productos, df_atributos, n)
            time.sleep(2)
            driver.back()
            time.sleep(3)
        except:
            return(df_productos, df_atributos, n)
    return(df_productos, df_atributos, n)

In [ ]:
def recorrer_producto(df_productos, n):
    photos = driver.find_elements_by_class_name('woocommerce-product-gallery__image')
    str_photos = ''
    for photo in photos:
        str_photos = str_photos + ',' + photo.find_element_by_tag_name('img').get_attribute('src')
    str_photos = str_photos[1:]
    title = driver.find_element_by_class_name('product_title.entry-title').text
    sku = driver.find_element_by_class_name('sku').text
    categories = driver.find_element_by_class_name('posted_in').text.replace(', ', ',').replace('CATEGORÍAS: ', '')
    tags = driver.find_element_by_class_name('tagged_as').text.replace(', ', ',').replace('ETIQUETAS: ', '')
    desc = driver.find_element_by_class_name('woocommerce-Tabs-panel.woocommerce-Tabs-panel--description.panel.entry-content.wc-tab').text
    #price
    producto = {'Product ID*':n,'Active (0/1)':1,'Name *':title,'Categories (x,y,z...)':categories,'Wholesale price':None, 'Reference #':sku, 'Description':desc, 'Tags (x,y,z...)':tags, 'Image URLs (x,y,z...)':str_photos}
    df_productos = df_productos.append(producto, ignore_index=True)
    return df_productos

In [104]:
pr, atr, num = recorrer_superproductos(df_productos, df_atributos, 0)

producto0
1
producto1
2
producto2
3
producto3
4
producto4
5


In [49]:
atr.to_csv('atributos_muebles1.xlsx', index = False, sep = ';', encoding = 'utf-8')

In [106]:
pr.to_csv('productos_luz.xlsx', index = False, sep = ';', encoding = 'utf-8')

In [105]:
pr

,Product ID*,Active (0/1),Name *,"Categories (x,y,z...)",Price tax excluded,Reference #,Description,"Tags (x,y,z...)","Image URLs (x,y,z...)"
0,1.0,1.0,FOCO ORO30 LED,CATEGORÍA: Iluminación,49.44,12580100,Foco Oro30 LED. Aplique LED 300 mm.,"Aplique,LED,Foco,iluminación",https://tegler.es/wp-content/uploads/2019/02/1...
1,2.0,1.0,FOCO NIKITA LED,CATEGORÍA: Iluminación,52.77,12520925,Foco Nikita Led. Modelo S-3R BLC LED CII IP44 ...,None,https://tegler.es/wp-content/uploads/2017/08/f...
2,3.0,1.0,FOCO VERÓNICA LED,CATEGORÍA: Iluminación,26.11,12520900,Foco VERÓNICA LED. Modelo VERÓNICA FOCO LED de...,None,https://tegler.es/wp-content/uploads/2017/08/1...


Prueba para averiguar colores (combinaciones)

In [2]:
valor = driver.find_elements_by_class_name('swatch.swatch-image')
for color in valor:
    print(color.find_element_by_tag_name('img').get_attribute('src'))
    print(color.text)
#colores, son importantes para clickar sobre ellos y sacar fotos

https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-crema-150x150.jpg
Crema
https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-negro-150x150.jpg
Negro
https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-blanco-150x150.jpg
Blanco
https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-chocolate-150x150.jpg
Chocolate
https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-grafito-150x150.jpg
Grafito
https://tegler.es/wp-content/uploads/2017/09/allstone-revestimiento-gris-cemento-150x150.jpg
Cemento


In [33]:
n = driver.find_elements_by_class_name('label')[0]
desplegable = Select(driver.find_element_by_id(n.find_element_by_tag_name('label').get_attribute('for')))
driver.execute_script('var items = {}; for (index = 0; index < arguments[0].attributes.length; ++index) { items[arguments[0].attributes[index].name] = arguments[0].attributes[index].value }; return items;', desplegable.options[1])

{'class': 'attached enabled', 'value': 'medida-100'}

In [64]:
n = driver.find_elements_by_class_name('label')[0]
desplegable = Select(driver.find_element_by_id(n.find_element_by_tag_name('label').get_attribute('for')))
desplegable

In [128]:
atr

,Product ID*,Attribute (Name:Type:Position)*,Value (Value:Position)*,Reference,Wholesale price,"Image URLs (x,y,z...)"
0,2.0,"Color:color:0, Medida:select:1","Crema:0, 80×150 cm.:1",11680150CR,466.91,https://tegler.es/wp-content/uploads/2017/09/a...
1,2.0,"Color:color:0, Medida:select:1","Crema:0, 80×160 cm.:1",11680160CR,490.05,https://tegler.es/wp-content/uploads/2017/09/a...
2,2.0,"Color:color:0, Medida:select:1","Negro:0, 80×150 cm.:1",11680150NG,466.91,https://tegler.es/wp-content/uploads/2017/09/a...
3,2.0,"Color:color:0, Medida:select:1","Negro:0, 80×160 cm.:1",11680160NG,490.05,https://tegler.es/wp-content/uploads/2017/09/a...
4,2.0,"Color:color:0, Medida:select:1","Blanco:0, 80×150 cm.:1",11680150BL,466.91,https://tegler.es/wp-content/uploads/2017/09/a...
5,2.0,"Color:color:0, Medida:select:1","Blanco:0, 80×160 cm.:1",11680160BL,490.05,https://tegler.es/wp-content/uploads/2017/09/a...
6,2.0,"Color:color:0, Medida:select:1","Chocolate:0, 80×150 cm.:1",11680150CH,466.91,https://tegler.es/wp-content/uploads/2017/09/a...
7,2.0,"Color:color:0, Medida:select:1","Chocolate:0, 80×160 cm.:1",11680160CH,490.05,https://tegler.es/wp-content/uploads/2017/09/a...
8,2.0,"Color:color:0, Medida:select:1","Grafito:0, 80×150 cm.:1",11680150GF,466.91,https://tegler.es/wp-content/uploads/2017/09/a...
9,2.0,"Color:color:0, Medida:select:1","Grafito:0, 80×160 cm.:1",11680160GF,490.05,https://tegler.es/wp-content/uploads/2017/09/a...


In [178]:
df_productos

,Product ID*,Active (0/1),Name *,"Categories (x,y,z...)",Wholesale price,Reference #,Description,"Tags (x,y,z...)","Image URLs (x,y,z...)"
0,1.0,1.0,PLATO DE DUCHA ARDESIA EXTRAPLANO ANTIDESLIZANTE,"Platos de ducha,Platos de ducha de resina",None,N/D,Plato de ducha Ardesia extraplano. Fabricado y...,"Plato de ducha Allstone,carga mineral,Plato de...",https://tegler.es/wp-content/uploads/2017/09/P...
1,2.0,1.0,PLATO DE DUCHA CON SEMI-MARCO-MODELO BIDAI,"Platos de ducha,Platos de ducha de resina",None,N/D,Plato de ducha con semi-marco-modelo Bidai dis...,"Plato de ducha resina,Plato de ducha con semi-...",https://tegler.es/wp-content/uploads/2018/06/P...
2,3.0,1.0,REVESTIMIENTO DE PARED PARA ARDESIA Y BIDAI,"Platos de ducha,Platos de ducha de resina",None,N/D,Revestimiento de pared. Son paneles de 5 mm de...,"revestimiento pared,revestimiento paredes,pare...",https://tegler.es/wp-content/uploads/2017/06/R...
3,4.0,1.0,PLATO DE DUCHA ENMARCADO – MODELO LOFT ¡NUEVO!,"Platos de ducha,Platos de ducha de resina",None,N/D,PLATO DE DUCHA ENMARCADO – MODELO LOFT ¡NUEVO!...,"Plato de ducha resina,Plato de ducha con carga...",https://tegler.es/wp-content/uploads/2020/01/P...
4,5.0,1.0,ORGANIZADOR DE DUCHA LOFT – EXCLUSIVO EN TEGLER,"Platos de ducha,Platos de ducha de resina",None,N/D,El organizador de ducha Loft es un artículo pa...,None,https://tegler.es/wp-content/uploads/2020/01/O...
5,1.0,1.0,PLATO DE DUCHA ARDESIA EXTRAPLANO ANTIDESLIZANTE,"Platos de ducha,Platos de ducha de resina",None,N/D,Plato de ducha Ardesia extraplano. Fabricado y...,"Plato de ducha Allstone,carga mineral,Plato de...",https://tegler.es/wp-content/uploads/2017/09/P...
6,2.0,1.0,PLATO DE DUCHA CON SEMI-MARCO-MODELO BIDAI,"Platos de ducha,Platos de ducha de resina",None,N/D,Plato de ducha con semi-marco-modelo Bidai dis...,"Plato de ducha resina,Plato de ducha con semi-...",https://tegler.es/wp-content/uploads/2018/06/P...
7,3.0,1.0,REVESTIMIENTO DE PARED PARA ARDESIA Y BIDAI,"Platos de ducha,Platos de ducha de resina",None,N/D,Revestimiento de pared. Son paneles de 5 mm de...,"revestimiento pared,revestimiento paredes,pare...",https://tegler.es/wp-content/uploads/2017/06/R...
8,4.0,1.0,PLATO DE DUCHA ENMARCADO – MODELO LOFT ¡NUEVO!,"Platos de ducha,Platos de ducha de resina",None,N/D,PLATO DE DUCHA ENMARCADO – MODELO LOFT ¡NUEVO!...,"Plato de ducha resina,Plato de ducha con carga...",https://tegler.es/wp-content/uploads/2020/01/P...
9,5.0,1.0,ORGANIZADOR DE DUCHA LOFT – EXCLUSIVO EN TEGLER,"Platos de ducha,Platos de ducha de resina",None,N/D,El organizador de ducha Loft es un artículo pa...,None,https://tegler.es/wp-content/uploads/2020/01/O...


In [37]:
pr['Price tax excluded'] = round(pr['Price tax excluded'],2)

In [38]:
pr

,Product ID*,Active (0/1),Name *,"Categories (x,y,z...)",Price tax excluded,Reference #,Description,"Tags (x,y,z...)","Image URLs (x,y,z...)",price tax excluded
0,1.0,1.0,OFERTA-INODORO Y BIDÉ RING SUSPENDIDOS – DISEÑ...,"Serie Ring,Ofertas,Complementos cerámicos,Sani...",105.79,13012140,OFERTA-INODORO Y BIDÉ RING SUSPENDIDOS – DISEÑ...,"Oferta baño,decoración baño,artículos para bañ...",https://tegler.es/wp-content/uploads/2020/11/O...,105.79
1,2.0,1.0,BIDÉ SUSPENDIDO RING – DISEÑO ITALIANO,"Serie Ring,Sanitarios,Bidés",45.45,13012135,BIDÉ SUSPENDIDO RING que presenta un diseño mo...,"Bidé,diseño italiano baños,bidé suspendido",https://tegler.es/wp-content/uploads/2020/11/B...,45.45
2,3.0,1.0,KIT INODORO SUSPENDIDO RING – DISEÑO ITALIANO,"Serie Ring,Sanitarios,Inodoros",87.60,13012130,Kit inodoro RING suspendido que presenta un di...,"Inodoro suspendido,diseño italiano baños",https://tegler.es/wp-content/uploads/2020/11/I...,87.60
3,4.0,1.0,KIT LEGEND INODORO + CISTERNA + ASIENTO DUROPLAST,"Sanitarios,Serie Legend,Inodoros",219.42,10035099,Kit de inodoro completo con sistema de salida ...,None,https://tegler.es/wp-content/uploads/2017/06/K...,219.42
4,5.0,1.0,BIDÉ LEGEND,"Sanitarios,Serie Legend,Bidés",127.77,10035120,Bidé cerámico blanco de la serie Legend.\nAguj...,None,https://tegler.es/wp-content/uploads/2017/06/B...,127.77
5,6.0,1.0,OFERTA-KIT DORENCE INODORO + CISTERNA + ASIENTO,"Ofertas,Sanitarios,Serie Dorence,Inodoros",195.04,10010319-OF,OFERTA-KIT DORENCE INODORO + CISTERNA + ASIENT...,"Duroplast,Sanitarios,cisterna,Inodoro completo...",https://tegler.es/wp-content/uploads/2020/09/O...,195.04
6,7.0,1.0,BIDÉ DORENCE,"Sanitarios,Serie Dorence,Bidés",108.32,10010340,Bidé cerámico blanco de la serie Dorence.\nAgu...,None,https://tegler.es/wp-content/uploads/2017/06/B...,108.32
7,8.0,1.0,OFERTA-KIT VIENA INODORO + CISTERNA + ASIENTO,"Ofertas,Sanitarios,Serie VIENA",157.00,10017200-OF,OFERTA-KIT VIENA INODORO + CISTERNA + ASIENTO....,"Inodoro completo,WC",https://tegler.es/wp-content/uploads/2020/10/O...,157.00
8,9.0,1.0,BIDÉ VIENA,"Sanitarios,Serie VIENA,Bidés",105.55,10017225,Bidé cerámico blanco de la serie Viena.\nAbert...,"Sanitarios,baño,Bidé",https://tegler.es/wp-content/uploads/2019/02/B...,105.55
9,10.0,1.0,OFERTA-KIT COMPACT INODORO + CISTERNA + ASIENTO,"Ofertas,Sanitarios,Serie Compact,Inodoros",156.20,10017134-OF,OFERTA-KIT COMPACT INODORO + CISTERNA + ASIENT...,"Inodoro,Sanitarios,WC",https://tegler.es/wp-content/uploads/2020/09/O...,156.20
